In [1]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import pandas as pd

In [2]:
# Load dataset as dataframe
train_df = pd.read_csv("txt_files/gender_train.txt", sep=' ', names=['datadir', 'gender'])
test_df = pd.read_csv("txt_files/gender_test.txt", sep=' ', names=['datadir', 'gender'])
train_df['datadir'] = 'data/aligned/' + train_df['datadir'].astype(str)
test_df['datadir'] = 'data/aligned/' + test_df['datadir'].astype(str)



In [8]:
print(train_df.shape)
print(test_df.shape)

(12256, 2)
(4007, 2)


In [7]:
train_df['gender'].value_counts() / 12256 * 100

1    54.569191
0    45.430809
Name: gender, dtype: float64

In [9]:
test_df['gender'].value_counts() / 4007 * 100

0    51.135513
1    48.864487
Name: gender, dtype: float64

In [ ]:

# Hyperparameters
batch_size = 128
# Load images into keras image generator 
datagen_train = ImageDataGenerator(rescale=1./255)
datagen_valid = ImageDataGenerator(rescale=1./255)
# For train generator
train_generator = datagen_train.flow_from_dataframe(
    dataframe = train_df,
    directory=None,
    x_col="datadir",
    y_col="gender",
    batch_size=batch_size,
    seed=42,
    shuffle=True,
    class_mode='raw',
    target_size=(224,224))
# For test generator 
valid_generator = datagen_valid.flow_from_dataframe(
    dataframe = test_df,
    directory=None,
    x_col="datadir",
    y_col="gender",
    batch_size=batch_size,
    seed=42,
    shuffle=True,
    class_mode='raw',
    target_size=(224,224))

In [5]:
model = Sequential()
model.add(ResNet50(include_top=False, pooling='avg', weights='imagenet'))
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(2048, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(1024, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(1, activation='softmax'))
model.layers[0].trainable = False
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 2048)              23587712  
_________________________________________________________________
flatten (Flatten)            (None, 2048)              0         
_________________________________________________________________
batch_normalization (BatchNo (None, 2048)              8192      
_________________________________________________________________
dense (Dense)                (None, 2048)              4196352   
_________________________________________________________________
batch_normalization_1 (Batch (None, 2048)              8192      
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              2098176   
_________________________________________________________________
batch_normalization_2 (Batch (None, 1024)              4

In [4]:
# Compile model 
model.compile(
    optimizer='Adam',
    loss='binary_crossentropy',
    metrics=['accuracy'],
)

In [5]:
model.fit(train_generator, epochs=10, validation_data=valid_generator)
# model.save_weights('second_model.h5') 

Epoch 1/10
 2/96 [..............................] - ETA: 11:03 - loss: 7.2672 - accuracy: 0.5234